<a href="https://www.kaggle.com/elakapoor/amazon-amazing-products?scriptVersionId=88448141" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/10000-amazon-products-dataset/Amazon_Products.csv


The objective is to find the best products at good price.

In [2]:
path = "/kaggle/input/10000-amazon-products-dataset/Amazon_Products.csv"
df = pd.read_csv(path, low_memory=False)
df.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,description,...,Unnamed: 885,Unnamed: 886,Unnamed: 887,Unnamed: 888,Unnamed: 889,Unnamed: 890,Unnamed: 891,Unnamed: 892,Unnamed: 893,Unnamed: 894
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby 2014 Catalogue Box ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Size Name:Large FunkyBuys® Large Christmas Hol...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby RailRoad 0-4-0 Gild...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# knowing the number of rows and columns
df.shape

(10004, 895)

In [4]:
# checking the missing values
df.isnull().sum()

uniq_id                          2
product_name                     2
manufacturer                     9
price                         1437
number_available_in_stock     2502
                             ...  
Unnamed: 890                 10002
Unnamed: 891                 10002
Unnamed: 892                 10002
Unnamed: 893                 10003
Unnamed: 894                 10003
Length: 895, dtype: int64

In [5]:
# droping the columns with 70% or more missing data
perc = 70.0 
min_count =  int(((100 - perc)/100) * df.shape[1] + 1)
mod_df = df.dropna(axis = 1, thresh = min_count)

In [6]:
# number of columns are reduced significantly from 895 to 14
mod_df.shape

(10004, 14)

In [7]:
# checking the column name
mod_df.columns

Index(['uniq_id', 'product_name', 'manufacturer', 'price',
       'number_available_in_stock', 'number_of_reviews',
       'number_of_answered_questions', 'average_review_rating',
       'amazon_category_and_sub_category', 'description',
       'product_information', 'product_description',
       'items_customers_buy_after_viewing_this_item',
       'customer_questions_and_answers'],
      dtype='object')

In [8]:
# checking the data type, missing values in remaining columns
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10004 entries, 0 to 10003
Data columns (total 14 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   uniq_id                                      10002 non-null  object
 1   product_name                                 10002 non-null  object
 2   manufacturer                                 9995 non-null   object
 3   price                                        8567 non-null   object
 4   number_available_in_stock                    7502 non-null   object
 5   number_of_reviews                            9984 non-null   object
 6   number_of_answered_questions                 9237 non-null   object
 7   average_review_rating                        9984 non-null   object
 8   amazon_category_and_sub_category             9312 non-null   object
 9   description                                  9351 non-null   object
 10  product_in

The column price, number_available_in_stock, number_of_reviews,number_of_answered_questions and average_review_rating have wrong datatype.
The datatype given is object but we want them to be int or float. Let us correct it.<br>
The column amazon_category_and_sub_category has multiple values. In order to clean the data we will seperate them to individual columns.<br>
We will also check the missing unique_id

In [9]:
# check the missing value in unique_id column
mod_df.dropna(subset=['uniq_id'], inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [10]:
# check for the length of each unique_id
code_len = mod_df["uniq_id"].map(lambda code: len(code))
code_len.unique()
# Here it seems some problem is present. let us check it
for i in mod_df["uniq_id"]:
    if len(i) == 1:
        print(i)
    
# here it seems that } is the unwanted entry. Thus we need to remove those rows
mod_df[mod_df["uniq_id"] == "}"]

# let us drop these rows

mod_df.drop([2003, 2005], inplace = True)

}
}


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
# number_available_in_stock can be new as well as used. 
# Let us check the if both type of items as present or not.
mod_df[mod_df['number_available_in_stock'].str.contains("used", na = False)].head(2)

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers
17,e9ef14a0253f074343b5441540d8471f,Chuggington Interactive Wash and Fuel Set with...,Chuggington,NaN,1 used,2,1,4.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description The Chuggington Interactiv...,Technical Details Item Weight998 g Product Dim...,Product Description The Chuggington Interactiv...,http://www.amazon.co.uk/Chuggington | http://w...,NaN
161,a8d29a2c05bb402335f29f433eb20beb,Hornby R322 OO Scale Class 8F 2-8-0 Steam Loco...,Hornby,NaN,1 used,1,2,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,An OO scale model of Stanier's heavy freight s...,Technical Details Manufacturer recommended age...,An OO scale model of Stanier's heavy freight s...,NaN,NaN


In [12]:
# creating new column to define the item type
item = ["new", "used"]
mod_df["item_type"] = mod_df['number_available_in_stock'].str.extract(f'({"|".join(item)})')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# Check if there is any other category that we have missed
mod_df["item_type"].unique()

array(['new', nan, 'used'], dtype=object)

In [14]:
# removing the pound sign
mod_df["price_new"] = mod_df["price"].str.split(" ", expand = True).get(0).str.split("£", expand = True).get(1)
# check if the value is in range of thousand
mod_df[mod_df.price_new.str.contains(',', na=False)]
# remove the comma sign
mod_df["price_new"] = mod_df["price_new"].str.replace(",", "")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
# extract the digits and change the type to float
mod_df["number_available_in_stock"] = mod_df["number_available_in_stock"].str.extract("(\d+)").astype(np.float)

# keep only number in the column
mod_df["average_review_rating(out of 5)"] = mod_df["average_review_rating"].str.split(" out of ", expand = True).get(0)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [16]:
# split the main category into multiple sub category
split_df = mod_df["amazon_category_and_sub_category"].str.split(">", expand = True)
mod_df["main_category"] = split_df.get(0)
mod_df["sub_category_1"] = split_df.get(1)
mod_df["sub_category_2"] = split_df.get(2)
mod_df["sub_category_3"] = split_df.get(3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [17]:
# change the type of column
mod_df["average_review_rating(out of 5)"] = mod_df["average_review_rating(out of 5)"].astype(float)
mod_df["price_new"] = mod_df["price_new"].astype(float)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# checking for the non numeric value
pd.to_numeric(mod_df["number_of_reviews"], errors='coerce')
pd.to_numeric(mod_df["number_of_reviews"], errors='coerce').isnull()
mod_df[pd.to_numeric(mod_df["number_of_reviews"], errors='coerce').isnull()]

# remove the comma sign
mod_df["number_of_reviews"] = mod_df["number_of_reviews"].str.replace(",", "")

# changing the type of column
mod_df["number_of_reviews"] = mod_df["number_of_reviews"].astype(float)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [19]:
# changing dtype of the column
mod_df["number_of_answered_questions"] = mod_df["number_of_answered_questions"].astype(float)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
# drop the columns not in use any more
mod_df.drop(["price", "average_review_rating", 
             "amazon_category_and_sub_category"],  axis=1, inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
# dataframe after first phase of cleaning
mod_df.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,5.0,15.0,1.0,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,new,3.42,4.9,Hobbies,Model Trains & Railway Sets,Rail Vehicles,Trains
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,NaN,2.0,1.0,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,NaN,16.99,4.5,Hobbies,Model Trains & Railway Sets,Rail Vehicles,Trains


In [22]:
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 10003
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      10000 non-null  object 
 1   product_name                                 10000 non-null  object 
 2   manufacturer                                 9993 non-null   object 
 3   number_available_in_stock                    7500 non-null   float64
 4   number_of_reviews                            9982 non-null   float64
 5   number_of_answered_questions                 9235 non-null   float64
 6   description                                  9349 non-null   object 
 7   product_information                          9941 non-null   object 
 8   product_description                          9348 non-null   object 
 9   items_customers_buy_after_viewing_this_item  6934 non-null   object 
 10

In [23]:
# plot the total missing values
x = mod_df.isnull().sum()

fig = px.bar(x, orientation = "h",  text_auto='.2s',
            color_discrete_sequence= ["#ff6b00"] * len(x))
fig.update_layout(
    title="<b>Missing Value Count</b>",
    xaxis_title="Total missing values",
    yaxis_title="Column Names",
    plot_bgcolor = "#ECECEC",
    showlegend=False
)
fig.show()

# EDA

Now let us imagine we are browsing the Amazon website. What are the things that you see when you click on a product. For me the priority order is as follows:
1. Price
2. Rating
3. Manufacturer
4. Description
5. Customer reviews<br>
Let us see analyze the given dataframe on following points.

In [24]:
# let us check and create a dataframe of missing price
missing_price = mod_df[mod_df["price_new"].isnull()]
missing_price.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
9,5afbaf65680c9f378af5b3a3ae22427e,Learning Curve Chuggington Interactive Chatsworth,Chuggington,1.0,8.0,1.0,Product Description An amazingly Interactive C...,Technical Details Item Weight150 g Product Dim...,Product Description An amazingly Interactive C...,http://www.amazon.co.uk/Chuggington | http://w...,NaN,new,NaN,4.8,Hobbies,Model Trains & Railway Sets,Rail Vehicles,Trains
14,c68c3ae3b0ea3146beae99f3d4a6997c,Kato 3060-2 EF65 500 (F Model) Electric Locomo...,Kato,18.0,1.0,1.0,Suitable for the following scale(s): N Scale,Technical Details Item Weight150 g Product Dim...,Suitable for the following scale(s): N Scale,NaN,NaN,new,NaN,5.0,Hobbies,Model Trains & Railway Sets,Rail Vehicles,Locomotives


In [25]:
# Since our further analysis is based on the price column so let us drop it.
df = mod_df.dropna(subset=['price_new'])
df.head()

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,5.0,15.0,1.0,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,new,3.42,4.9,Hobbies,Model Trains & Railway Sets,Rail Vehicles,Trains
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,NaN,2.0,1.0,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,NaN,16.99,4.5,Hobbies,Model Trains & Railway Sets,Rail Vehicles,Trains
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,2.0,17.0,2.0,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,new,9.99,3.9,Hobbies,Model Trains & Railway Sets,Rail Vehicles,Trains
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,NaN,1.0,2.0,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,NaN,39.99,5.0,Hobbies,Model Trains & Railway Sets,Rail Vehicles,Trains
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,NaN,3.0,2.0,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,NaN,32.19,4.7,Hobbies,Model Trains & Railway Sets,Rail Vehicles,Trains


In [26]:
# let us check the manufactures according to their prices
df[["price_new", "manufacturer"]].groupby("manufacturer").max().sort_values(by = "price_new",
                                                                    ascending = False)

,price_new
manufacturer,
Bowen Designs,2439.92
DJI,995.11
Sideshow,719.95
AUTOart,648.95
Bushiroad,629.95
...,...
k2-accessories Acrylic Plastic Beads,0.90
Card Concept,0.83
unisex party bag fillers,0.60


In [27]:
# detail of the maximum price row
df[df["price_new"] == df["price_new"].max()]

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
7098,75d96583894e50412ba5c0161375fef5,Bowen Designs - Marvel Statue Thor Classic Act...,Bowen Designs,2.0,1.0,1.0,Sideshow Collectibles is proud to present the ...,Technical Details Item Weight6 Kg Product Dime...,Sideshow Collectibles is proud to present the ...,NaN,NaN,new,2439.92,4.0,Hobbies,Model Trains & Railway Sets,Accessories,Figures & Decoration Parts


In [28]:
# detail of the minimum price row
df[df["price_new"] == df["price_new"].min()]

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
6366,30950ba413fbae88b190efb5c7237fc2,Happy 18th Birthday Foil Banner - 365cm,Partyrama,NaN,1.0,1.0,Happy 18th Birthday Foil Banner - Sold Single....,Technical Details Manufacturer recommended age...,Happy 18th Birthday Foil Banner - Sold Single....,http://www.amazon.co.uk/18th-Happy-Birthday-La...,NaN,NaN,0.01,5.0,Party Supplies,"Banners, Stickers & Confetti",Banners,None


In [29]:
# let us check the common manufacture
values = df["manufacturer"].value_counts().keys().tolist()[:10]
counts = df["manufacturer"].value_counts().tolist()[:10]

In [30]:
fig = px.bar(df, y = counts, x = values,
            color_discrete_sequence = ["#EC2781"] * len(df))


fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  yaxis_title = "Count",
                xaxis_title = "Name of Manufacturers",
                  title = "<b>Popular Manufacturers Category</b>"
                 )
fig.show()

From above graph we see that the Oxfor Diecast is most popular. Let us check the main category for the above top 10 brands

In [31]:
# creating the dataframe of top 10 manufacturer
df_list = []
for i in values:
    x = df[df["manufacturer"] == i]
    df_list.append(x)
frame = pd.concat(df_list)
frame.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
804,d994e8cb4d0b15683da91652901d9895,Oxford Diecast Anglia Police Panda 76105003,Oxford Diecast,4.0,4.0,11.0,"The Ford Anglia , appeared in 1959 and when it...",Technical Details Item Weight18 g Product Dime...,"The Ford Anglia , appeared in 1959 and when it...",http://www.amazon.co.uk/Cararama-Anglia-Sunbur...,NaN,new,4.75,4.8,Die-Cast & Toy Vehicles,Toy Vehicles & Accessories,Scaled Models,Vehicles
830,8f4a9ace7bb4ae6e6d8dda96d94bd332,Oxford Diecast Evening News Metal Model,Oxford Diecast,5.0,1.0,1.0,Product Description In the early 1960s the mos...,Technical Details Item Weight82 g Product Dime...,Product Description In the early 1960s the mos...,http://www.amazon.co.uk/Omnibus-RM075-OMNIBUS-...,NaN,new,10.95,5.0,Die-Cast & Toy Vehicles,Toy Vehicles & Accessories,Scaled Models,Vehicles


In [32]:
# average rating of the manufactures
frame[["manufacturer", "average_review_rating(out of 5)"]].groupby("manufacturer").mean().sort_values(by = "average_review_rating(out of 5)",
                                                ascending = False)

,average_review_rating(out of 5)
manufacturer,
MyTinyWorld,4.923656
Oxford Diecast,4.790789
Corgi,4.783333
The Puppet Company,4.759804
Star Wars,4.738710
LEGO,4.736232
Playmobil,4.726496
Disney,4.687500
Mattel,4.661039


Even though the most popular brand is Oxford Diecast but the highest rated is MyTinyWorld.<br>
Also the manufacturer MyTinyWorld is sixth most favourite in the popular manufacturer category. On the other hand Oxford Diecast is second in terms of average rating<br>
Let us now check the popular main category items present.

In [33]:
# different main categories present
frame["main_category"].unique()

array(['Die-Cast & Toy Vehicles ', nan, 'Characters & Brands ',
       'Figures & Playsets ', 'Hobbies ', 'Party Supplies ',
       'Electronic Toys ', 'Arts & Crafts ', 'Sports Toys & Outdoor ',
       'Dolls & Accessories ', 'Puppets & Puppet Theatres ',
       'Cooking & Dining ', 'Games '], dtype=object)

In [34]:
fig = px.bar(frame, "main_category", 
             color_discrete_sequence = ["#2377a4"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  yaxis_title = "Count",
                  xaxis_title = "Main Categories",
                  title = "<b>Count of Main Categories of Products</b>"
                 )
fig.show()

From the graph we see that Character & Brands is the popular main category. Let us select the top 10 popular main category. We are narrowing our selction to reach the goal

In [35]:
# let us select the 5 popular main categories

value_main = frame["main_category"].value_counts().keys().tolist()[:5]
count_main = frame["main_category"].value_counts().tolist()[:5]
value_main

['Characters & Brands ',
 'Die-Cast & Toy Vehicles ',
 'Figures & Playsets ',
 'Dolls & Accessories ',
 'Puppets & Puppet Theatres ']

Let us create a new dataframe having top 10 popular manufcturers and 5 most popular main category.

In [36]:
df_list = []
for i in value_main:
    x = frame[frame["main_category"] == i]
    df_list.append(x)
    #print(df)
frame = pd.concat(df_list)
frame.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
627,15e03b806b3d90bced2186049c4f3613,LEGO Star Wars 75018: Jek-14's Stealth Starfig...,LEGO,16.0,30.0,1.0,Product Description LEGO® Star Wars™ Jek-14’s ...,Technical Details Item Weight907 g Product Dim...,Product Description LEGO® Star Wars™ Jek-14’s ...,http://www.amazon.co.uk/Star-Wars-75102-Fighte...,NaN,new,68.87,4.8,Characters & Brands,Star Wars,Toys,None
651,ec2b46b7363449a91b66a0e721ae4b9a,LEGO Star Wars 75015: Corporate Alliance Tank ...,LEGO,21.0,34.0,2.0,Product Description LEGO® Star Wars™ Corporate...,Technical Details Item Weight299 g Product Dim...,Product Description LEGO® Star Wars™ Corporate...,http://www.amazon.co.uk/LEGO-Star-Wars-75135-I...,what other figures can I buy to go with this s...,new,39.99,4.8,Characters & Brands,Star Wars,Toys,None


In [37]:
# let us check the popular subcategory

import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
frame_sub = frame[["main_category", "sub_category_1", "sub_category_2", "sub_category_3"]].groupby("main_category").count()
frame_sub.style.background_gradient(cmap=cm)
# As sub_category_3 consist of very less entries let us drop it.


,sub_category_1,sub_category_2,sub_category_3
main_category,,,
Characters & Brands,327,287,0
Die-Cast & Toy Vehicles,245,244,230
Dolls & Accessories,103,93,0
Figures & Playsets,130,0,0
Puppets & Puppet Theatres,99,0,0


In [38]:
value_sub_1 = frame["sub_category_1"].value_counts().keys().tolist()[:10]
count_sub_1 = frame["sub_category_1"].value_counts().tolist()[:10]

value_sub_2 = frame["sub_category_2"].value_counts().keys().tolist()[:5]
count_sub_2 = frame["sub_category_2"].value_counts().tolist()[:5]

In [39]:
# new dataframe with selected sub_category_1
df_list = []
for i in value_sub_1:
    x = frame[frame["sub_category_1"] == i]
    df_list.append(x)
frame = pd.concat(df_list)
frame.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
804,d994e8cb4d0b15683da91652901d9895,Oxford Diecast Anglia Police Panda 76105003,Oxford Diecast,4.0,4.0,11.0,"The Ford Anglia , appeared in 1959 and when it...",Technical Details Item Weight18 g Product Dime...,"The Ford Anglia , appeared in 1959 and when it...",http://www.amazon.co.uk/Cararama-Anglia-Sunbur...,NaN,new,4.75,4.8,Die-Cast & Toy Vehicles,Toy Vehicles & Accessories,Scaled Models,Vehicles
830,8f4a9ace7bb4ae6e6d8dda96d94bd332,Oxford Diecast Evening News Metal Model,Oxford Diecast,5.0,1.0,1.0,Product Description In the early 1960s the mos...,Technical Details Item Weight82 g Product Dime...,Product Description In the early 1960s the mos...,http://www.amazon.co.uk/Omnibus-RM075-OMNIBUS-...,NaN,new,10.95,5.0,Die-Cast & Toy Vehicles,Toy Vehicles & Accessories,Scaled Models,Vehicles


In [40]:
# new dataframe with selected sub_category_2

df_list = []
for i in value_sub_2:
    x = frame[frame["sub_category_2"] == i]
    df_list.append(x)
    #print(df)
frame = pd.concat(df_list)
frame.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
2574,a1210448395e60572d10add1eb03952c,LEGO Star Wars Force Awakens First Order Speci...,LEGO,9.0,1.0,1.0,Intercept the enemy with the impressive LEGO S...,Technical Details Manufacturer recommended age...,Intercept the enemy with the impressive LEGO S...,http://www.amazon.co.uk/LEGO-Star-Wars-75101-S...,NaN,new,67.98,5.0,Characters & Brands,Disney,Toys,None
309,c375310e315e2763112e32d6b4a6f244,DESPICABLE ME 2 - Minion cuddly Soft Toy - Plu...,Disney,2.0,1.0,1.0,Colour Name:Bob DESPICABLE ME 2 - Minion cuddl...,Technical Details Item Weight222 g Product Dim...,Colour Name:Bob DESPICABLE ME 2 - Minion cuddl...,http://www.amazon.co.uk/Minion-Looks-BANANA-MI...,NaN,new,19.99,5.0,Characters & Brands,Disney,Toys,None


Now we have completed the second phase of data preprocessing. After this we have achieved a dataframe with following characterstics:
1. No null price.
2. Top 10 manufacturers with respect to count
3. Popular top 5 main categories, 10 sub_category_1 and 5 sub_category_2 <br>
Let us now check the average rating and price for this selected dataframe.

In [41]:
# rating of the products
print("The average rating: ",frame["average_review_rating(out of 5)"].unique())

# After processing our data we have significantly reduced the size of the dataframe.
# Also the rating are now 4 or greater.
# Let us now check new average price ### check above before processing to compare.
print("The new average price: ", frame["price_new"].mean())



The average rating:  [5.  4.1 4.4 4.5 4.  4.3 4.2 4.8 4.6 4.7 4.9]
The new average price:  16.697757774140705


In [42]:
# let us check the item type
print("Does this new data have used items as well: ", frame["item_type"].value_counts())
print("There are still used item in our top selected dataframe. Let us check these items.")
# still we have some used item. let us check these used item
frame_used = frame.loc[frame["item_type"] == "used"]
frame_used.head(2)

Does this new data have used items as well:  new     443
used      7
Name: item_type, dtype: int64
There are still used item in our top selected dataframe. Let us check these items.


,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3
4223,d0a6a1575f7306aed1514d9e3cb16ceb,Disney Pixar - CARS - SUPERCHARGED - Die-Cast ...,Disney,1.0,1.0,2.0,Disney Pixar - CARS - SUPERCHARGED - Die-Cast ...,Technical Details Item Weight1.1 Kg Product Di...,Disney Pixar - CARS - SUPERCHARGED - Die-Cast ...,NaN,NaN,used,66.64,5.0,Characters & Brands,Disney,Toys,None
6718,4fa1f79a926c91f17aa68232ee5f4d51,Disney Princess Art Case,Disney,1.0,6.0,4.0,This great Disney Princess decorated art case ...,Technical Details Item Weight440 g Product Dim...,This great Disney Princess decorated art case ...,http://www.amazon.co.uk/Disney-Princess-Colour...,NaN,used,7.99,4.7,Characters & Brands,Disney,Toys,None


In [43]:
# they have an average of 4.5 which is very good
frame_used["average_review_rating(out of 5)"].mean()

4.5285714285714285

In [44]:
import plotly.figure_factory as ff
x = frame["price_new"]
hist_data = [x]
group_labels = ['price']

fig = ff.create_distplot(hist_data, group_labels, show_rug = False,
                        colors=["#ffd514"])
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  title = "<b>Price Distribution of Data</b>"
                 )

fig.show()

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



We see that the plot is right skew plot with the presence of outliers. Let us see these outliers

In [45]:
# check the statistics of the price_new column
frame.price_new.describe()

count    611.000000
mean      16.697758
std       22.145520
min        0.960000
25%        5.950000
50%       10.370000
75%       20.120000
max      230.110000
Name: price_new, dtype: float64

In [46]:
# plot the quartiles and check for outliers 
fig = px.box(frame, "price_new")
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  title = "<b>Price Data Distribution</b>",
                 xaxis_title = "Price of Products"
                 )
fig.show()

In [47]:
# let us find the outliers
Q1 = 5.95
Q2 = 10.37
Q3 = 20.12
IQR = Q3 - Q1
outlier1 = (Q1 - 1.5 * IQR)
outlier2 = (Q3 + 1.5 * IQR)
print("outlier1: ", outlier1)
print("outlier2: ", outlier2)

outlier1:  -15.305000000000003
outlier2:  41.375


In [48]:
outlier_price = []

for i in frame.price_new:
    if i < outlier1 or i > outlier2:
        outlier_price.append("outlier")
    elif i > outlier1 or i < outlier2:
        outlier_price.append("normal")
    
frame["outlier_price"] = outlier_price

In [49]:
fig = px.pie(frame, names = frame["outlier_price"], color = frame["outlier_price"],
             color_discrete_map={'normal': '#2377a4', 'outlier': '#ffd514'})

fig.update_layout(title = "<b>Distribution of Outlier</b>")

fig.show()

In [50]:
# let us see the outlier value
frame_outlier = frame.loc[frame["outlier_price"] == "outlier"].head()
frame_outlier

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3,outlier_price
2574,a1210448395e60572d10add1eb03952c,LEGO Star Wars Force Awakens First Order Speci...,LEGO,9.0,1.0,1.0,Intercept the enemy with the impressive LEGO S...,Technical Details Manufacturer recommended age...,Intercept the enemy with the impressive LEGO S...,http://www.amazon.co.uk/LEGO-Star-Wars-75101-S...,NaN,new,67.98,5.0,Characters & Brands,Disney,Toys,None,outlier
4223,d0a6a1575f7306aed1514d9e3cb16ceb,Disney Pixar - CARS - SUPERCHARGED - Die-Cast ...,Disney,1.0,1.0,2.0,Disney Pixar - CARS - SUPERCHARGED - Die-Cast ...,Technical Details Item Weight1.1 Kg Product Di...,Disney Pixar - CARS - SUPERCHARGED - Die-Cast ...,NaN,NaN,used,66.64,5.0,Characters & Brands,Disney,Toys,None,outlier
4306,541c64951229414f1207855fa78770f6,Disney Pixar Cars - Mega Mack Raceworld Playset,Disney,3.0,14.0,2.0,Mega Mack from Disney Pixar's Cars has an amaz...,Technical Details Item Weight1.8 Kg Product Di...,Mega Mack from Disney Pixar's Cars has an amaz...,http://www.amazon.co.uk/Disney-Pixar-Cars-Syst...,NaN,new,57.02,4.2,Characters & Brands,Disney,Toys,None,outlier
6894,9354e13bec4b477d3b8edec4ada7ea96,LEGO Pirates of the Caribbean 4181: Isla De la...,Disney,32.0,17.0,1.0,Manufacturer's Description Pirate Lord Hector ...,Technical Details Item Weight9 g Product Dimen...,Manufacturer's Description Pirate Lord Hector ...,http://www.amazon.co.uk/LEGO-Pirates-Caribbean...,NaN,new,68.30,4.8,Characters & Brands,Disney,Toys,None,outlier
8072,c12cbd7a1dfbdc2c06765d2d6b6a4837,Toy Story 3 Collection Roundup Interactive TAL...,Disney,8.0,3.0,1.0,Toy Story 3 Interactive Collection - Woody's H...,Technical Details Item Weight1.4 Kg Product Di...,Toy Story 3 Interactive Collection - Woody's H...,http://www.amazon.co.uk/Toy-Story-Woodys-Horse...,NaN,new,54.99,5.0,Characters & Brands,Disney,Toys,None,outlier


In [51]:
print("Manufacturers with outlier price: ", frame_outlier.manufacturer.value_counts())

Manufacturers with outlier price:  Disney    4
LEGO      1
Name: manufacturer, dtype: int64


In [52]:
# top category
print("Main category with outliers: ", frame_outlier.main_category.value_counts())

Main category with outliers:  Characters & Brands     5
Name: main_category, dtype: int64


In [53]:
print("Sub_category_2 with outliers: ",frame_outlier.sub_category_2.value_counts())

Sub_category_2 with outliers:   Toys    5
Name: sub_category_2, dtype: int64


In [54]:
# let us check if any used item is present
print("Item_type with outlier price: ", frame_outlier.item_type.unique())
# even used item is present 

Item_type with outlier price:  ['new' 'used']


In [55]:
# check the information of used item type present with outlier price
frame_outlier[frame_outlier["item_type"] == "used"]

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3,outlier_price
4223,d0a6a1575f7306aed1514d9e3cb16ceb,Disney Pixar - CARS - SUPERCHARGED - Die-Cast ...,Disney,1.0,1.0,2.0,Disney Pixar - CARS - SUPERCHARGED - Die-Cast ...,Technical Details Item Weight1.1 Kg Product Di...,Disney Pixar - CARS - SUPERCHARGED - Die-Cast ...,NaN,NaN,used,66.64,5.0,Characters & Brands,Disney,Toys,None,outlier


In [56]:
# Let us check the rating of the products
fig = px.violin(frame, "average_review_rating(out of 5)", 
               color_discrete_sequence = ["#FFBF00"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  xaxis_title = "Rating",
                  title = "<b>Rating Distribution of the Popular Products</b>"
                 )
fig.show()

As expected our selected category of products have the most common rating as 5. It also seems that our selection of top manufacturer, categories have resulted in dataframe having minimum rating as 3.8. Now let us check the customer reviews

In [57]:
fig = px.histogram(frame, "number_of_reviews",
                  color_discrete_sequence = ["#8B4000"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  xaxis_title = "Number of Reviews",
                  title = "<b>Number of Reviews Distribution</b>"
                 )
fig.show()

In [58]:
# Let us check if there are any null review
print("Number of null values: ",frame['number_of_reviews'].isnull().sum())
# it seems that with high end products people love to leave a review

Number of null values:  0


In [59]:
frame.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),main_category,sub_category_1,sub_category_2,sub_category_3,outlier_price
2574,a1210448395e60572d10add1eb03952c,LEGO Star Wars Force Awakens First Order Speci...,LEGO,9.0,1.0,1.0,Intercept the enemy with the impressive LEGO S...,Technical Details Manufacturer recommended age...,Intercept the enemy with the impressive LEGO S...,http://www.amazon.co.uk/LEGO-Star-Wars-75101-S...,NaN,new,67.98,5.0,Characters & Brands,Disney,Toys,None,outlier
309,c375310e315e2763112e32d6b4a6f244,DESPICABLE ME 2 - Minion cuddly Soft Toy - Plu...,Disney,2.0,1.0,1.0,Colour Name:Bob DESPICABLE ME 2 - Minion cuddl...,Technical Details Item Weight222 g Product Dim...,Colour Name:Bob DESPICABLE ME 2 - Minion cuddl...,http://www.amazon.co.uk/Minion-Looks-BANANA-MI...,NaN,new,19.99,5.0,Characters & Brands,Disney,Toys,None,normal


In [60]:
fig = px.scatter(frame, x="number_available_in_stock", y="number_of_reviews", 
                 trendline="ols")

fig.update_layout(title = "<b>Relationship between the number of reviews and product stock</b>",
                 plot_bgcolor = "#ECECEC",
                 yaxis_title = "Number of reviews",
                 xaxis_title = "Number available in stock")

fig.show()

In [61]:
fig = px.histogram(frame, "number_available_in_stock",
                  color_discrete_sequence = ["#C04000"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  xaxis_title = "Products Available in Stocks",
                  title = "<b>Number of Products Available in Stocks</b>"
                 )
fig.show()

From above analysis of the selected frame of popular categories we arrive at following conclusions:
1. The products with price less than 50 are popular.
2. The outlier in price data are around 6.55%
3. Disney and LEGO are the manufactures with outlier price
4. Character and brand is the subcatogory with outlier price
5. Even used item is also present with the outlier price
6. The maximum number rating of popular brands is in range of 5 star
7. Mostly 0-4 review were given on the products.
8. Every product has a review in selected dataframe
9. Mostly 2-3 products are present in the stock<br>
We can analyse at other relationship as well but for now the above is the conclusion.
If you like above notebook do upvote and leave your thoughts in comment section!!